In [19]:
#resnet_transf.py 
#Faz transfer learning usando ResNet50.
#https://medium.com/abraia/first-steps-with-transfer-learning-for-custom-image-classification-with-keras-b941601fcad5
import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import numpy as np;
from tensorflow import keras; import keras.backend as K;
from keras import optimizers, callbacks, regularizers;
from keras.regularizers import l2;
from keras.models import Sequential, Model;
from keras.layers import Dropout, Conv2D, MaxPooling2D, Dense, Flatten;
from keras.applications.resnet import ResNet50, preprocess_input, decode_predictions
from keras.utils import load_img, img_to_array

In [20]:
def leCsv(nomeDir, nomeArq, nl=0, nc=0):
    st=os.path.join(nomeDir,nomeArq);
    arq=open(st,"rt"); lines=arq.readlines(); arq.close(); n=len(lines)
    linhas_separadas=[]
    for linha in lines:
       linha=linha.strip('\n'); linha=linha.split(';'); linhas_separadas.append(linha);
    ay=np.empty((n),dtype='float32'); ax=np.empty((n,nl,nc,3),dtype='float32');
    for i in range(len(linhas_separadas)):
       linha=linhas_separadas[i];
       img_path=os.path.join(nomeDir,linha[0])
       t = load_img(img_path, target_size=(nl,nc))
       x = img_to_array(t)
       x = np.expand_dims(x, axis=0)
       ax[i] = preprocess_input(x)
       ay[i] = np.float32(linha[1]); #0=m ou 1=f
    return ax, ay


In [21]:
#<<<<<<<<<<<<<<<<<<<<<<<< main <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
nomeprog="resnet_transf"
#Original: 280x200, redimensionado: 224x224
num_classes=2; nl=224; nc=224
diretorioBd="Dados"
ax, ay = leCsv(diretorioBd,"treino.csv", nl=nl, nc=nc); #200 imagens
qx, qy = leCsv(diretorioBd,"teste.csv", nl=nl, nc=nc); #100 imagens
vx, vy = leCsv(diretorioBd,"valida.csv", nl=nl, nc=nc); #100 imagens
ay = keras.utils.to_categorical(ay, num_classes)
qy = keras.utils.to_categorical(qy, num_classes)
vy = keras.utils.to_categorical(vy, num_classes)

In [22]:
input_shape = (nl,nc,3); batch_size = 10;
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
# base_model.summary()
x = base_model.output
x = Flatten()(x)
avg_pool_2d = keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')
x = avg_pool_2d(100, activation="relu")(x)
predictions = Dense(num_classes, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [23]:
#Nao permite treinar base_model. So as camadas densas sao treinadas:
for layer in base_model.layers: layer.trainable = False
#Treina com learning rate grande
otimizador=keras.optimizers.Adam(learning_rate=1e-3)
model.compile(otimizador, loss='categorical_crossentropy', metrics =['accuracy'])
model.fit(ax, ay, batch_size=batch_size, epochs=5, verbose=2, validation_data=(vx,vy))

Epoch 1/5


KeyError: 'model_1/conv2_block1_2_conv/Conv2D'

In [ ]:
score = model.evaluate(ax, ay, verbose=0); print('Training loss:', score)
score = model.evaluate(vx, vy, verbose=0); print('Validation loss:', score)
score = model.evaluate(qx, qy, verbose=0); print('Test loss:', score)

In [ ]:
#Libera todos layers do model para treinar:
for layer in model.layers: layer.trainable = True
#Treina com learning rate pequena todas as camadas
otimizador=keras.optimizers.Adam(learning_rate=1e-6)
model.compile(otimizador, loss='categorical_crossentropy', metrics =['accuracy'])
model.fit(ax, ay, batch_size=batch_size, epochs=5, verbose=2, validation_data=(vx,vy))

In [ ]:
score = model.evaluate(ax, ay, verbose=0); print('Training loss:', score)
score = model.evaluate(vx, vy, verbose=0); print('Validation loss:', score)
score = model.evaluate(qx, qy, verbose=0); print('Test loss:', score)
model.save(nomeprog+".h5")
